# Instalando Dependecias

In [ ]:
pip install rdflib

In [ ]:
pip install pandas

In [ ]:
pip install SPARQLWrapper

In [ ]:
pip install pyvis

# Importaciones

In [ ]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import pandas as pd

In [ ]:
import pyvis
from pyvis.network import Network
import networkx as nx
import json

# Desarrollo

## 1. Filtrando los datos de la película (usando SPARQL)

In [ ]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
def detallePelicula(titulo):
    sparql.setQuery('''
        SELECT ?name ?director ?country ?starring ?abstract ?studio ?producer ?music
        WHERE { dbr:'''+titulo+''' dbp:name ?name .
                dbr:'''+titulo+''' dbo:director ?director .
                dbr:'''+titulo+''' dbp:country ?country .
                dbr:'''+titulo+''' dbp:starring ?starring .
                dbr:'''+titulo+''' dbo:abstract ?abstract .
                dbr:'''+titulo+''' dbp:studio ?studio .
                dbr:'''+titulo+''' dbp:producer ?producer .
                dbr:'''+titulo+''' dbp:music ?music .

                FILTER (lang(?abstract) = 'en')
        }''')
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    
    diccionarios = []
    for res in qres["results"]["bindings"]:
        dic ={
            "name":res["name"]["value"],
            "director":res["director"]["value"].split("/")[-1],
            "country":res["country"]["value"],
            "abstract":res["abstract"]["value"],
            "starring":res["starring"]["value"].split("/")[-1],
            "studio":res["studio"]["value"].split("/")[-1],
            "producer":res["producer"]["value"].split("/")[-1],
            "music":res["music"]["value"].split("/")[-1],
        }
        diccionarios.append(dic)
    print(len(diccionarios))
    
    dics_string = json.dumps(diccionarios)  
    a_json = json.loads(dics_string)
    dataframe = pd.DataFrame.from_dict(a_json)
    dataframe.drop_duplicates()
    
    return dataframe

In [ ]:
detallePelicula('Are_We_Done_Yet%3F')

In [ ]:
detallePelicula('Toy_Story_3')

## Relación Actor - Película - País de Origen - Estudio - Productor - Productor Musical

In [ ]:
def grafo(df):
    G = nx.Graph()

    for a in df.itertuples():
        if a.starring != "":
            G.add_node(a.starring, size="20",labelHighlightBold=True)
            G.add_edge(a.starring, a.name)
            G.add_edge(a.name, a.country)
            G.add_edge(a.name, a.studio)
            G.add_edge(a.name, a.producer)
            G.add_edge(a.name, a.music)

    
    net = Network(notebook=True,directed=True)

    net.from_nx(G)
    #net.show_buttons()
    net.repulsion(node_distance=150, spring_length=100,central_gravity=0.5)
    return net.show("example.html")

In [ ]:
def nuevo(titulo):
    dataframe = detallePelicula(titulo)
    dataframe.reset_index()
    dataframe.drop_duplicates()
    return dataframe

def enriquecer(dataframe,titulo):
    df=dataframe.append(detallePelicula(titulo))
    df.reset_index()
    df.drop_duplicates()
    return df

In [ ]:
dataframe = nuevo("Toy_Story_3")
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"Cars_2")
dataframe.head(100)
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"The_Good_Dinosaur")
dataframe.head(100)
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"Wild_Hogs")
dataframe.head(100)
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"Are_We_Done_Yet%3F")
dataframe.head(100)
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"Forrest_Gump")
dataframe.head(100)
grafo(dataframe)

In [ ]:
dataframe = enriquecer(dataframe,"Lucky_Numbers")
dataframe.head(100)
grafo(dataframe)